In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_columns', None)

# Leitura do .csv usando a biblioteca do Python pandas
df = pd.read_csv("../input/aula-2-ia-dataset/CasasParaAlugar.csv", index_col = 0)

# FEATURES_ANALISAR = ['city', 'area', 'rooms', 'bathroom', 'parking spaces', 'floor', 
#                      'animal', 'furniture', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 
#                      'fire insurance (R$)', 'total (R$)']

# df = df.loc[:, FEATURES_ANALISAR]
# df

# Tamanho do dataset
print("{} linhas\n{} colunas/features.".format(df.shape[0], df.shape[1]))

################# Medição, deleção e imputação de dados ausentes ################# 

# # Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0)

y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Quantidade de variáveis ausentes por coluna")
plt.show()


######################### LIMPEZA DO DATASET #########################
########### Imputação de dados por Regressão Linear (apenas para colunas numéricas) ########### 

# Criamos um dataframe com os dados das respectivas colunas:
# 'area', 'rooms', 'bathroom', 'parking spaces', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)'
df_imput_regress = pd.concat([df['area'], df['rooms'], df['bathroom'], df['parking spaces'], df['hoa (R$)'], df['rent amount (R$)'], df['property tax (R$)'], df['fire insurance (R$)'], df['total (R$)']], axis=1)
df_imput_regress.head()

## Aplicação da Regressão Linear ##

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state = 0)

# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_imput_regress.values)

# Agora, faremos uma regressão nos mesmos dados usados no treinamento, para
# gerar valores numéricos para substituir os valores ausentes de LotFrontage
X = df_imput_regress.values
regr_output = imp_mean.transform(X)
regr_output

## Substituição dos dados treinados para o dataset real:

df['area'] = regr_output[:, 0]
df['rooms'] = regr_output[:, 1]
df['bathroom'] = regr_output[:, 2]
df['parking spaces'] = regr_output[:, 3]
df['hoa (R$)'] = regr_output[:, 4]
df['rent amount (R$)'] = regr_output[:, 5]
df['property tax (R$)'] = regr_output[:, 6]
df['fire insurance (R$)'] = regr_output[:, 7]
df['total (R$)'] = regr_output[:, 8]

# df

## Confirmação que não restam mais valores nulos nas colunas
df['area'].isna().sum()
df['rooms'].isna().sum()
df['bathroom'].isna().sum()
df['parking spaces'].isna().sum()
df['hoa (R$)'].isna().sum()
df['rent amount (R$)'].isna().sum()
df['property tax (R$)'].isna().sum()
df['fire insurance (R$)'].isna().sum()
df['total (R$)'].isna().sum()

########### Imputação de dados via maior frequência (apenas para colunas nominais) ########### 
# NominalCategorical = ['city', 'animal', 'furniture']

df['city'].value_counts()
df['animal'].value_counts()
df['furniture'].value_counts()
df['floor'].value_counts()


#.fillna substitui o argumento nos dados ausentes
df['city'] = df['city'].fillna('São Paulo')
df['animal'] = df['animal'].fillna('acept')
df['furniture'] = df['furniture'].fillna('not furnished')
df['floor'] = df['floor'].fillna('1')

# Substituindo o "-" na coluna floor por "0"
df['floor'] = df['floor'].str.replace('-','0')
# df

########### OUTLIERS ###########

NominalCategorical = ['city', 'animal', 'furniture']
OrdinalCategorical = ['floor'] 
Numeric = ['area', 'rooms', 'bathroom', 'parking spaces', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']

## Detecção visual

####### TESTE 1
selected_columns = Numeric + OrdinalCategorical
df[selected_columns].head()

fig, ax = plt.subplots()

ax.scatter(x=df['rooms'], y=df['bathroom'])
plt.show()

###### TESTE 2
selected_columns = Numeric + OrdinalCategorical
df[selected_columns].head()

fig, ax = plt.subplots()

# Valores acima de 50.000 seram excluídos 
ax.scatter(x=df['property tax (R$)'], y=df['fire insurance (R$)'])
plt.show()

print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['property tax (R$)'] < 50000
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

# df

## FEATURE ENGINEERING -> SOMA DO RENT AMOUNT + PROPERTY TAX = coluna "amount total rent"

df['amount total rent'] = df['rent amount (R$)'] + df['property tax (R$)']
df[['amount total rent', 'rent amount (R$)', 'property tax (R$)']]

## FEATURE SELECTION -> Descartar uma feature usando o teste f-regression

# df

colunasSelecionadas = ['floor','area', 'rooms', 'bathroom', 'parking spaces', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']

x = df[colunasSelecionadas]
y = df['total (R$)']


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

# k é o numero de features que NÃO serão jogadas foras. Vamos primeiro ver os resultados, depois eliminar alguma feature.
k = x.shape[1]
# Utilizamos um método do sklearn para isso, usando a estratégia Chi Squared.
selector = SelectKBest(f_regression, k=k)
x_new = selector.fit_transform(x, y)

# Utilizo o log10 pois os valores são ou muito grandes, ou muito pequenos
scores = -np.log10(selector.pvalues_)

x_plot = list(range(len(scores)))

fig, ax = plt.subplots(figsize=(8, 4))
plt.bar(x_plot, scores)
ax.set_title("Score do método f-regression para Feature Selection")
ax.set_xticks(x_plot)
ax.set_xticklabels(selected_columns, rotation=45)
plt.show()

## A feature "area" apresenta os menores resultados, e elimina-se a mesma do dataset

print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

# df = df.drop(['area'], axis=1)

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))


### FEATURE ENCODING ###
# Furniture possui dois valores: "furnished" or "not furnished"

df['furniture'].value_counts()

df['furniture'].replace(to_replace='furnished', value=1, inplace=True)
df['furniture'].replace(to_replace='not furnished', value=0, inplace=True)

df['furniture'].value_counts()

######## DATA SCALING #######

colunas = ['floor','area', 'rooms', 'bathroom', 'parking spaces', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']


df[colunas].describe()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = df[colunas]
scaler.fit(data)

# Reescalonamos os dados
data_scaled = scaler.transform(data)

# Criamos um dataframe para facilitar a visualização
data_scaled = pd.DataFrame(data_scaled)
# "Devolvemos" os nomes das features e os índices para o dataframe
data_scaled.columns = colunas
data_scaled.index = df.index

data_scaled.describe()

df = df.drop(colunas, axis=1)
df = pd.concat([df, data_scaled], axis=1)

df







